# Ćwiczenie: Wdrożenie DataBota z uwierzytelnieniem użytkowników

**Czas trwania:** 45 minut

**Cel ćwiczenia:** W tym ćwiczeniu stworzymy prostą aplikację webową udostępniającą DataBota użytkownikom, skonfigurujemy uwierzytelnianie użytkowników z wykorzystaniem Azure AD/EntraID i zintegrujemy backend DataBota z interfejsem.

## 1. Wprowadzenie do udostępniania modeli LLM w organizacji

Aby modele językowe, takie jak nasz DataBot, mogły być efektywnie wykorzystywane w organizacji, musimy zapewnić odpowiedni interfejs dostępu dla użytkowników końcowych. Istnieje kilka kluczowych aspektów, które należy wziąć pod uwagę:

- **Metody dostępu**: REST API, aplikacje webowe, integracja z aplikacjami biurowymi (Teams, Slack)
- **Bezpieczeństwo**: Uwierzytelnianie użytkowników, autoryzacja dostępu do danych, szyfrowanie komunikacji
- **Wydajność**: Optymalizacja kosztów, równoważenie obciążenia, buforowanie często zadawanych pytań
- **UX (User Experience)**: Intuicyjny interfejs, obsługa historii konwersacji, formatowanie odpowiedzi
- **Monitorowanie**: Śledzenie użycia, analiza zapytań, zbieranie feedbacku od użytkowników

W tym ćwiczeniu skupimy się na tworzeniu aplikacji webowej z wykorzystaniem Streamlit, która zapewni przyjazny interfejs dla DataBota, oraz na implementacji uwierzytelniania użytkowników przy użyciu Azure AD/EntraID.

## 2. Przygotowanie środowiska dla aplikacji

Zamiast lokalnie instalować środowisko i uruchamiać backend, sugerujemy:

### 🔧 Wariant 1 – uruchomienie backendu jako job w Databricks:
1. Przejdź do zakładki **Workflows** > **Jobs**.
2. Utwórz nowy Job z notatnikiem zawierającym backend lub FastAPI.
3. Ustaw parametr `uvicorn app:app --host 0.0.0.0 --port 8000` jeśli używasz `%sh`.

### 🔧 Wariant 2 – wdrożenie frontendu poza Databricks (zalecane):
1. Użyj **Azure App Service** lub **Static Web Apps**.
2. Sekrety i endpointy mogą być zarządzane przez Databricks Secrets + REST API.


In [ ]:
# ⛔️ Instalacja ręczna bibliotek
# %pip install fastapi uvicorn msal

Zamiast instalacji lokalnej, dodaj biblioteki do klastra Databricks:

1. Przejdź do zakładki **Compute** > wybierz swój klaster.
2. Kliknij **Libraries** > **Install New** > **PyPI**.
3. Wpisz nazwę biblioteki, np. `fastapi`, `uvicorn`, `msal`.


## 3. Rejestracja aplikacji w Azure AD/EntraID

Aby aplikacja frontendowa mogła korzystać z modelu, należy zarejestrować ją w Azure:

### 🔧 Kroki:
1. Przejdź do [portal.azure.com](https://portal.azure.com), wybierz **Microsoft Entra ID**.
2. Wybierz **App registrations** → **New registration**.
3. Wprowadź nazwę i URI aplikacji (np. http://localhost:8000 lub URL aplikacji w Azure).
4. Po rejestracji:
   - Skopiuj `Application (client) ID`
   - Skonfiguruj `Redirect URI`
   - Utwórz `Client secret` i zapisz go jako sekret w Databricks (`Secrets` → `Create Secret Scope`).

Model może być chroniony przez weryfikację tokena przesłanego przez klienta.


In [ ]:
# Konfiguracja poświadczeń aplikacji Azure AD
# W środowisku produkcyjnym te wartości powinny być przechowywane w Azure Key Vault lub Databricks Secrets

# Wartości z rejestracji aplikacji w Azure AD
CLIENT_ID = "YOUR_APPLICATION_CLIENT_ID"  # ID aplikacji
CLIENT_SECRET = "YOUR_APPLICATION_CLIENT_SECRET"  # Sekret aplikacji
TENANT_ID = "YOUR_TENANT_ID"  # ID katalogu (tenant)
AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
REDIRECT_URI = "http://localhost:8501"  # URI przekierowania, taki sam jak przy rejestracji aplikacji

# Zakres uprawnień
SCOPE = ["https://graph.microsoft.com/.default"]

# Zapisanie jako zmienne środowiskowe
os.environ["AZURE_CLIENT_ID"] = CLIENT_ID
os.environ["AZURE_CLIENT_SECRET"] = CLIENT_SECRET
os.environ["AZURE_TENANT_ID"] = TENANT_ID
os.environ["AZURE_AUTHORITY"] = AUTHORITY
os.environ["AZURE_REDIRECT_URI"] = REDIRECT_URI

print("Skonfigurowano poświadczenia aplikacji Azure AD")

## 4. Tworzenie frontendu DataBota z użyciem Streamlit

Streamlit to biblioteka Python, która umożliwia szybkie tworzenie interaktywnych aplikacji webowych. Jest idealna do prototypowania interfejsów dla modeli ML/AI.

In [ ]:
# Przykładowy kod frontendu Streamlit
# Plik: databot_app.py

streamlit_code = '''
import streamlit as st
import requests
import json
import os
import uuid
from msal import ConfidentialClientApplication, PublicClientApplication
import time

# Konfiguracja aplikacji
st.set_page_config(
    page_title="Enterprise DataBot",
    page_icon="🤖",
    layout="wide"
)

# Parametry uwierzytelniania Azure AD
CLIENT_ID = os.environ.get("AZURE_CLIENT_ID")
CLIENT_SECRET = os.environ.get("AZURE_CLIENT_SECRET")
TENANT_ID = os.environ.get("AZURE_TENANT_ID")
AUTHORITY = os.environ.get("AZURE_AUTHORITY")
REDIRECT_URI = os.environ.get("AZURE_REDIRECT_URI")
SCOPE = ["https://graph.microsoft.com/.default"]

# Endpoint DataBota (można zastąpić rzeczywistym endpointem API)
DATABOT_API_ENDPOINT = "http://localhost:5000/api/databot/query"

# Funkcja do uwierzytelniania użytkownika
def authenticate_user():
    # Inicjalizacja aplikacji MSAL
    app = ConfidentialClientApplication(
        client_id=CLIENT_ID,
        client_credential=CLIENT_SECRET,
        authority=AUTHORITY
    )
    
    # Generowanie URL do logowania
    auth_url = app.get_authorization_request_url(
        scopes=SCOPE,
        state=str(uuid.uuid4()),
        redirect_uri=REDIRECT_URI
    )
    
    # Przekierowanie użytkownika do logowania
    st.markdown(f"[Login with Microsoft]({auth_url})")
    
    # Oczekiwanie na kod autoryzacji (w rzeczywistej aplikacji byłoby to obsługiwane przez backend)
    auth_code = st.text_input("Po zalogowaniu wklej kod autoryzacji z URL:", type="password")
    
    if auth_code:
        try:
            # Wymiana kodu autoryzacji na token dostępu
            result = app.acquire_token_by_authorization_code(
                code=auth_code,
                scopes=SCOPE,
                redirect_uri=REDIRECT_URI
            )
            
            if "access_token" in result:
                st.session_state["access_token"] = result["access_token"]
                st.session_state["user_info"] = result.get("id_token_claims", {})
                st.success("Zalogowano pomyślnie!")
                time.sleep(1)  # Krótkie opóźnienie przed odświeżeniem
                st.experimental_rerun()  # Odświeżenie strony
            else:
                st.error(f"Błąd uwierzytelniania: {result.get('error_description', 'Nieznany błąd')}")
        except Exception as e:
            st.error(f"Wystąpił błąd: {str(e)}")

# Funkcja do wysyłania zapytań do DataBota
def query_databot(query, access_token):
    # W rzeczywistej aplikacji tutaj byłoby wywołanie API DataBota
    # Na potrzeby demonstracji używamy symulowanej odpowiedzi
    
    # Symulacja opóźnienia odpowiedzi
    with st.spinner("DataBot przetwarza zapytanie..."):
        time.sleep(1.5)  # Symulacja czasu przetwarzania
    
    # Symulowana odpowiedź
    if "azure" in query.lower() or "microsoft" in query.lower():
        return "Azure to chmura obliczeniowa Microsoftu oferująca szeroki zakres usług, w tym obliczenia, analitykę, przechowywanie i sieci. Azure umożliwia tworzenie, wdrażanie i zarządzanie aplikacjami w globalnej sieci centrów danych Microsoft."
    elif "databricks" in query.lower():
        return "Databricks to ujednolicona platforma analityczna oparta na Apache Spark, która umożliwia zespołom współpracę nad danymi, uczeniem maszynowym i analityką w jednym miejscu. Łączy najlepsze cechy hurtowni danych i jezior danych w jedną architekturę Lakehouse."
    elif "databot" in query.lower():
        return "DataBot to inteligentny asystent AI, który łączy modele językowe z dostępem do danych organizacyjnych. Wykorzystuje architekturę RAG (Retrieval Augmented Generation) do dostarczania odpowiedzi opartych na wewnętrznych danych firmy."
    else:
        # Domyślna odpowiedź dla innych zapytań
        return f"Zrozumiałem twoje pytanie: '{query}'. W rzeczywistej implementacji DataBot otrzymałby odpowiedź z backendu, który wykonałby wyszukiwanie w danych organizacyjnych i wykorzystał model LLM do wygenerowania odpowiedzi."

# Główny interfejs aplikacji
def main():
    # Nagłówek aplikacji
    st.title("Enterprise DataBot 🤖")
    st.markdown("---")
    
    # Inicjalizacja stanu sesji, jeśli nie istnieje
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []
    
    if "access_token" not in st.session_state:
        # Użytkownik nie jest zalogowany, pokaż formularz logowania
        st.subheader("Zaloguj się, aby korzystać z Enterprise DataBot")
        authenticate_user()
    else:
        # Użytkownik jest zalogowany, pokaż interfejs chatu
        user_info = st.session_state.get("user_info", {})
        user_name = user_info.get("name", "Użytkownik")
        
        # Panel boczny z informacjami o użytkowniku
        with st.sidebar:
            st.subheader("Profil użytkownika")
            st.write(f"Zalogowano jako: **{user_name}**")
            
            # Przycisk wylogowania
            if st.button("Wyloguj"):
                # Usunięcie danych sesji przy wylogowaniu
                for key in ["access_token", "user_info", "chat_history"]:
                    if key in st.session_state:
                        del st.session_state[key]
                st.experimental_rerun()
            
            st.markdown("---")
            st.subheader("O aplikacji")
            st.info(
                "Enterprise DataBot to inteligentny asystent AI, który łączy modele językowe z "
                "dostępem do danych organizacyjnych. Możesz zadawać pytania dotyczące firmowych "
                "dokumentów, danych i procedur."
            )
        
        # Główny obszar chatu
        chat_container = st.container()
        
        # Wyświetlenie historii chatu
        with chat_container:
            for i, (role, text) in enumerate(st.session_state.chat_history):
                if role == "user":
                    st.markdown(f"**👤 {user_name}:** {text}")
                else:  # role == "assistant"
                    st.markdown(f"**🤖 DataBot:** {text}")
                
                # Dodanie separatora między wiadomościami, ale nie po ostatniej
                if i < len(st.session_state.chat_history) - 1:
                    st.markdown("---")
        
        # Formularz do nowego zapytania
        with st.form(key="query_form"):
            query = st.text_area("Zadaj pytanie DataBotowi:", height=100)
            submit_button = st.form_submit_button(label="Wyślij")
            
            if submit_button and query.strip():
                # Dodanie zapytania użytkownika do historii
                st.session_state.chat_history.append(("user", query))
                
                # Wysłanie zapytania do DataBota
                access_token = st.session_state["access_token"]
                response = query_databot(query, access_token)
                
                # Dodanie odpowiedzi DataBota do historii
                st.session_state.chat_history.append(("assistant", response))
                
                # Odświeżenie strony, aby wyświetlić nowe wiadomości
                st.experimental_rerun()

# Uruchomienie aplikacji
if __name__ == "__main__":
    main()
'''

# Zapisanie kodu do pliku
app_file_path = "databot_app.py"
with open(app_file_path, "w") as f:
    f.write(streamlit_code)

print(f"Kod aplikacji Streamlit został zapisany do pliku: {app_file_path}")

Przeanalizujmy główne elementy kodu aplikacji:

1. **Konfiguracja aplikacji Streamlit**: Ustawienia strony, tytuł, ikona i układ
2. **Uwierzytelnianie użytkownika**: Wykorzystanie biblioteki MSAL do uwierzytelniania z Azure AD
3. **Obsługa sesji**: Przechowywanie tokenu dostępu i informacji o użytkowniku w st.session_state
4. **Interfejs chatu**: Panel boczny z informacjami o użytkowniku i główny obszar konwersacji
5. **Przetwarzanie zapytań**: Funkcja query_databot do wysyłania zapytań i odbierania odpowiedzi

## 5. Tworzenie backendu do obsługi zapytań DataBota

Teraz stworzymy backend, który będzie odpowiedzialny za przetwarzanie zapytań od frontendu, weryfikację tokenu uwierzytelniania i komunikację z modelem DataBota.

In [ ]:
# Przykładowy kod backendu z Flask
# Plik: databot_api.py

flask_code = '''
from flask import Flask, request, jsonify
import jwt
import requests
import os
import json
from functools import wraps

# Inicjalizacja aplikacji Flask
app = Flask(__name__)

# Konfiguracja Azure AD
CLIENT_ID = os.environ.get("AZURE_CLIENT_ID")
TENANT_ID = os.environ.get("AZURE_TENANT_ID")
JWKS_URI = f"https://login.microsoftonline.com/{TENANT_ID}/discovery/v2.0/keys"

# Funkcja pobierająca klucze publiczne z Azure AD
def get_jwks():
    response = requests.get(JWKS_URI)
    if response.status_code == 200:
        return response.json()["keys"]
    else:
        raise Exception(f"Nie udało się pobrać kluczy JWT: {response.status_code}")

# Dekorator do weryfikacji tokenu JWT
def verify_token(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        # Pobranie tokenu z nagłówka Authorization
        auth_header = request.headers.get("Authorization")
        if not auth_header or not auth_header.startswith("Bearer "):
            return jsonify({"error": "Brak tokenu uwierzytelniającego"}), 401
        
        token = auth_header.split(" ")[1]
        
        try:
            # Dekodowanie tokenu bez weryfikacji (by pobrać kid)
            header = jwt.get_unverified_header(token)
            
            # Pobranie klucza publicznego odpowiadającego kid
            jwks = get_jwks()
            public_key = None
            for key in jwks:
                if key["kid"] == header["kid"]:
                    public_key = jwt.algorithms.RSAAlgorithm.from_jwk(json.dumps(key))
                    break
            
            if not public_key:
                return jsonify({"error": "Nie znaleziono odpowiedniego klucza publicznego"}), 401
            
            # Weryfikacja tokenu
            payload = jwt.decode(
                token,
                public_key,
                algorithms=["RS256"],
                audience=CLIENT_ID,
                options={"verify_exp": True}
            )
            
            # Dodanie informacji o użytkowniku do kontekstu żądania
            request.user = payload
            
            return f(*args, **kwargs)
        except jwt.ExpiredSignatureError:
            return jsonify({"error": "Token wygasł"}), 401
        except jwt.InvalidTokenError as e:
            return jsonify({"error": f"Nieprawidłowy token: {str(e)}"}), 401
    
    return decorated

# Symulacja modelu DataBota (w rzeczywistej aplikacji tutaj byłoby wywołanie modelu)
def process_query(query, user_info):
    # W rzeczywistej implementacji tutaj byłoby wywołanie modelu DataBota
    # Na potrzeby demonstracji zwracamy symulowaną odpowiedź
    user_name = user_info.get("name", "użytkownik")
    
    if "azure" in query.lower() or "microsoft" in query.lower():
        return f"Cześć {user_name}! Azure to chmura obliczeniowa Microsoftu oferująca szeroki zakres usług, w tym obliczenia, analitykę, przechowywanie i sieci. Azure umożliwia tworzenie, wdrażanie i zarządzanie aplikacjami w globalnej sieci centrów danych Microsoft."
    elif "databricks" in query.lower():
        return f"Cześć {user_name}! Databricks to ujednolicona platforma analityczna oparta na Apache Spark, która umożliwia zespołom współpracę nad danymi, uczeniem maszynowym i analityką w jednym miejscu. Łączy najlepsze cechy hurtowni danych i jezior danych w jedną architekturę Lakehouse."
    elif "databot" in query.lower():
        return f"Cześć {user_name}! DataBot to inteligentny asystent AI, który łączy modele językowe z dostępem do danych organizacyjnych. Wykorzystuje architekturę RAG (Retrieval Augmented Generation) do dostarczania odpowiedzi opartych na wewnętrznych danych firmy."
    else:
        # Domyślna odpowiedź dla innych zapytań
        return f"Cześć {user_name}! Zrozumiałem twoje pytanie: '{query}'. W rzeczywistej implementacji DataBot wykonałby wyszukiwanie w danych organizacyjnych i wykorzystał model LLM do wygenerowania odpowiedzi."

# Endpoint do przetwarzania zapytań
@app.route("/api/databot/query", methods=["POST"])
@verify_token
def query_databot():
    # Pobranie danych z żądania
    data = request.json
    if not data or "query" not in data:
        return jsonify({"error": "Brak zapytania w żądaniu"}), 400
    
    query = data["query"]
    user_info = request.user
    
    # Przetworzenie zapytania przez model DataBota
    response = process_query(query, user_info)
    
    return jsonify({
        "response": response,
        "user": user_info.get("name", "unknown"),
        "timestamp": "2023-06-20T12:34:56Z"  # W rzeczywistej aplikacji byłby aktualny czas
    })

# Endpoint healthcheck
@app.route("/health", methods=["GET"])
def health_check():
    return jsonify({"status": "ok"})

# Uruchomienie aplikacji
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)
'''

# Zapisanie kodu do pliku
api_file_path = "databot_api.py"
with open(api_file_path, "w") as f:
    f.write(flask_code)

print(f"Kod API backendu został zapisany do pliku: {api_file_path}")

Kluczowe elementy backendu to:

1. **Weryfikacja tokenów JWT**: Bezpieczna weryfikacja tokenów używanych do uwierzytelniania użytkowników
2. **Pobieranie informacji o użytkowniku**: Ekstrakcja danych o użytkowniku z tokenu JWT
3. **Przetwarzanie zapytań**: Funkcja process_query, która w rzeczywistej implementacji wywoływałaby model DataBota
4. **Endpoint REST API**: /api/databot/query do obsługi zapytań od frontendu
5. **Healthcheck**: Prosty endpoint /health do monitorowania stanu aplikacji

## 6. Integracja frontendu z backendem DataBota

Teraz zmodyfikujemy naszą aplikację Streamlit, aby komunikowała się z backendem Flask.

In [ ]:
# Zmodyfikowany fragment kodu Streamlit do integracji z backendem
# Plik: databot_app_integrated.py

integration_code = '''
# Funkcja do wysyłania zapytań do API DataBota
def query_databot(query, access_token):
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    data = {
        "query": query
    }
    
    try:
        # Wysłanie zapytania do backendu
        with st.spinner("DataBot przetwarza zapytanie..."):
            response = requests.post(
                DATABOT_API_ENDPOINT,
                headers=headers,
                json=data,
                timeout=30  # Timeout w sekundach
            )
        
        if response.status_code == 200:
            result = response.json()
            return result["response"]
        elif response.status_code == 401:
            st.error("Sesja wygasła. Proszę zalogować się ponownie.")
            # Usunięcie tokenu z sesji
            if "access_token" in st.session_state:
                del st.session_state["access_token"]
            st.experimental_rerun()
            return None
        else:
            st.error(f"Błąd API: {response.status_code} - {response.text}")
            return f"Przepraszam, wystąpił błąd podczas przetwarzania zapytania (kod {response.status_code})."
    except requests.exceptions.RequestException as e:
        st.error(f"Błąd połączenia z API: {str(e)}")
        return "Przepraszam, nie mogę połączyć się z backendem DataBota. Proszę spróbować ponownie później."
'''

print("Fragment kodu integracji frontendu z backendem")
print(integration_code)

Ten fragment kodu zastępuje symulowaną funkcję `query_databot` w naszej aplikacji Streamlit. Nowa wersja:

1. Wysyła żądanie HTTP POST do backendu z tokenem dostępu w nagłówku Authorization
2. Obsługuje różne kody odpowiedzi (200 - sukces, 401 - brak autoryzacji, inne - błąd)
3. Wyświetla informacje o postępie i błędach użytkownikowi
4. Obsługuje problemy z połączeniem i limity czasowe

## 7. Wdrożenie aplikacji w Databricks

Aplikację możemy wdrożyć na kilka sposobów, ale jednym z najprostszych jest użycie Databricks Apps.

In [ ]:
# Instrukcje wdrażania aplikacji w Databricks
deployment_instructions = """
### Wdrażanie aplikacji w Databricks

#### Metoda 1: Wdrażanie przez interfejs użytkownika
1. Przejdź do workspace Databricks
2. Wybierz sekcję "Repos" (Repozytoria)
3. Utwórz nowe repozytorium lub użyj istniejącego
4. Prześlij pliki aplikacji (databot_app.py i inne) do repozytorium
5. Przejdź do sekcji "Apps" (Aplikacje)
6. Kliknij "Create App" (Utwórz aplikację)
7. Wybierz typ aplikacji: Streamlit
8. Wskaż ścieżkę do głównego pliku aplikacji (databot_app.py)
9. Skonfiguruj zasoby (rozmiar klastra)
10. Kliknij "Create" (Utwórz)

#### Metoda 2: Wdrażanie przez API Databricks (przykładowy kod)
"""

print(deployment_instructions)

# Przykładowy kod do wdrażania aplikacji przez API Databricks
deployment_code = '''
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import apps

# Inicjalizacja klienta Workspace
client = WorkspaceClient()

# Definicja aplikacji
app_definition = apps.AppCreate(
    name="DataBot",
    app_type="STREAMLIT",
    file_path="/Repos/your_username/databot_app/databot_app.py",
    cluster_settings=apps.ClusterSettings(
        cluster_size="Small"
    )
)

# Utworzenie aplikacji
try:
    app = client.apps.create(app_definition)
    print(f"Aplikacja utworzona pomyślnie. App ID: {app.app_id}")
    
    # Pobranie URL aplikacji
    app_info = client.apps.get(app.app_id)
    print(f"URL aplikacji: {app_info.app_url}")
except Exception as e:
    print(f"Błąd podczas tworzenia aplikacji: {str(e)}")
'''

print(deployment_code)

## 8. Filtrowanie danych po roli użytkownika

Ważnym aspektem DataBota w organizacji jest kontrola dostępu do danych. Poniżej przykład implementacji mechanizmu filtrowania danych w zależności od roli użytkownika.

In [ ]:
# Przykładowy kod do filtrowania danych po roli użytkownika
# Plik: databot_access_control.py

access_control_code = '''
# Definicja uprawnień dla różnych ról
ROLE_PERMISSIONS = {
    "User": ["read:public"],
    "Analyst": ["read:public", "read:department", "read:analytics"],
    "Admin": ["read:public", "read:department", "read:analytics", "read:confidential", "write:data"]
}

# Funkcja do pobierania roli użytkownika z tokenów
def get_user_role(user_info):
    # W rzeczywistej aplikacji role byłyby pobierane z tokenów lub z bazy danych
    # Na potrzeby demonstracji przypisujemy role na podstawie identyfikatora użytkownika
    user_id = user_info.get("oid", "")  # object ID użytkownika w Azure AD
    
    # Przykładowe mapowanie identyfikatorów użytkowników na role
    # W rzeczywistej aplikacji byłoby to pobierane z bazy danych lub serwisu autoryzacji
    user_roles = {
        "user1-id": "Admin",
        "user2-id": "Analyst",
        # Domyślna rola dla pozostałych użytkowników
        "default": "User"
    }
    
    return user_roles.get(user_id, user_roles["default"])

# Funkcja do sprawdzania, czy użytkownik ma uprawnienia do konkretnego zasobu
def has_permission(user_info, required_permission):
    role = get_user_role(user_info)
    permissions = ROLE_PERMISSIONS.get(role, [])
    return required_permission in permissions

# Przykład użycia w procesie przetwarzania zapytania
def process_query_with_access_control(query, user_info):
    # Określenie wymaganych uprawnień na podstawie treści zapytania
    required_permission = "read:public"  # Domyślne uprawnienie
    
    # Analiza zapytania pod kątem wrażliwych danych
    if any(keyword in query.lower() for keyword in ["poufne", "tajne", "confidential", "sensitive"]):
        required_permission = "read:confidential"
    elif any(keyword in query.lower() for keyword in ["finanse", "przychody", "budżet", "finance", "revenue"]):
        required_permission = "read:analytics"
    
    # Sprawdzenie uprawnień
    if not has_permission(user_info, required_permission):
        return "Przepraszam, nie masz uprawnień do dostępu do tych informacji. Skontaktuj się z administratorem, aby uzyskać dostęp."
    
    # Przetworzenie zapytania (tutaj byłoby wywołanie właściwego modelu DataBota)
    # ...
    
    # Przykładowa odpowiedź
    return f"Przetworzono zapytanie z uprawnieniem {required_permission}. W rzeczywistej aplikacji DataBot dostarczyłby odpowiedzi dostosowane do Twojego poziomu dostępu."
'''

# Zapisanie kodu do pliku
access_control_file_path = "databot_access_control.py"
with open(access_control_file_path, "w") as f:
    f.write(access_control_code)

print(f"Kod kontroli dostępu został zapisany do pliku: {access_control_file_path}")

Ten kod implementuje mechanizm kontroli dostępu, który:

1. Definiuje różne role użytkowników i ich uprawnienia
2. Przypisuje role użytkownikom na podstawie ich identyfikatorów
3. Sprawdza, czy użytkownik ma wymagane uprawnienia do konkretnych danych
4. Analizuje treść zapytania, aby określić wymagany poziom dostępu
5. Zwraca odpowiednie odpowiedzi w zależności od uprawnień użytkownika

## 9. Lokalne uruchomienie aplikacji

Dla celów testowych i demonstracyjnych możemy uruchomić aplikację lokalnie.

In [ ]:
# Instrukcje uruchomienia aplikacji lokalnie (dla celów demonstracyjnych)
local_run_instructions = """
### Uruchomienie aplikacji lokalnie (dla celów demonstracyjnych)

#### Frontend (Streamlit)
```bash
streamlit run databot_app.py
```

#### Backend (Flask)
```bash
python databot_api.py
```

Po uruchomieniu obu aplikacji, frontend Streamlit będzie dostępny pod adresem http://localhost:8501
"""

print(local_run_instructions)

Pamiętaj, że w środowisku produkcyjnym powinniśmy używać zarządzanych usług, takich jak Azure App Service, Azure Container Apps lub Databricks Apps, które zapewniają lepsze bezpieczeństwo, skalowalność i niezawodność.

## 10. Zadania do wykonania

1. Zaimplementuj mechanizm błędnej obsługi tokenów (np. wygasanie, odświeżanie)
2. Dodaj możliwość eksportu historii konwersacji do pliku
3. Zaimplementuj mechanizm oceny odpowiedzi przez użytkowników (thumbs up/down)
4. Rozszerz kontrolę dostępu o dodatkowe role i uprawnienia

## Zadania dodatkowe

Jeśli masz więcej czasu, możesz spróbować:

1. Dodanie funkcjonalności śledzenia historii konwersacji w bazie danych
2. Implementacja mechanizmu oceny odpowiedzi przez użytkowników (feedback)
3. Dodanie wizualizacji danych w odpowiedziach DataBota
4. Implementacja mechanizmu eksportu konwersacji do pliku

## Źródła i zasoby dodatkowe

Poniżej znajdują się linki do dodatkowych zasobów, które mogą być przydatne:

1. [Dokumentacja Microsoft Identity Platform](https://learn.microsoft.com/pl-pl/azure/active-directory/develop/)
2. [Wprowadzenie do Streamlit](https://docs.streamlit.io/library/get-started)
3. [Dokumentacja Databricks Apps](https://docs.databricks.com/applications/index.html)
4. [Przewodnik po uwierzytelnianiu w Azure](https://learn.microsoft.com/pl-pl/azure/active-directory/authentication/overview-authentication)
5. [Wzorce projektowe dla usług AI](https://learn.microsoft.com/pl-pl/azure/architecture/patterns/)